## 1. Expression of $ Z(\alpha, \beta)$

The partition function $ Z(\alpha, \beta) $ ensures that $ p(x) $ is a valid probability distribution:

$$
Z(\alpha, \beta) = \sum_{x \in \{0,1\}^n} \exp \left( \alpha \sum_i x_i + \beta \sum_{i \sim j} \mathbf{1}_{x_i = x_j} \right)
$$

where the sum runs over all possible configurations \( x \) of the system.

### Why is $ Z(\alpha, \beta)$ difficult to compute?
- The sum has **exponential complexity** in $ n $.
- Computing it exactly requires summing over $ 2^n $ configurations, which is infeasible for large $ n $.
- Instead, we use **sampling methods** (like Monte Carlo methods) to approximate it.






## Likelihood and Log-Likelihood 

Let $ x^1, x^2, \dots, x^n $ represent the **different observed configurations** of the Ising model, where each $ x^i $ is a vector of binary values representing the spin configurations.

### Likelihood Function

The **likelihood** is the **product of the probabilities** of these observed configurations given the model parameters $ \alpha $ and $ \beta $. For each configuration $ x^i $, the probability is given by the Ising model distribution:

$$
p(x^i \mid \alpha, \beta) = \frac{1}{Z(\alpha, \beta)} \exp \left( \alpha \sum_j x^i_j + \beta \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} \right)
$$

The **likelihood** is the product of the probabilities for all observed configurations $ x^1, x^2, \dots, x^n $:

$$
L(\alpha, \beta \mid x^1, x^2, \dots, x^n) = \prod_{i=1}^{n} p(x^i \mid \alpha, \beta)
$$

Substituting the probability for each observation:

$$
L(\alpha, \beta \mid x^1, x^2, \dots, x^n) = \prod_{i=1}^{n} \frac{1}{Z(\alpha, \beta)} \exp \left( \alpha \sum_j x^i_j + \beta \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} \right)
$$

### Log-Likelihood Function

To simplify the computation, we take the **logarithm** of the likelihood to obtain the **log-likelihood**:

$$
\log L(\alpha, \beta \mid x^1, x^2, \dots, x^n) = \sum_{i=1}^{n} \left( \alpha \sum_j x^i_j + \beta \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} \right) - n \log Z(\alpha, \beta)
$$

Where:
- $ x^i_j $ refers to the $ j $-th element (spin) of the $ i $-th configuration vector $ x^i $.
- The sum $ \sum_j x^i_j $ is the total sum of spins in the $ i $-th configuration.
- The sum $ \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} $ counts the number of neighboring pairs of spins that are equal in the $ i $-th configuration.
- $ Z(\alpha, \beta) $ is the partition function, which normalizes the distribution.

---

### Why is MLE difficult?
- Computing $ Z(\alpha, \beta)$ is **intractable**.
- Gradient-based optimization is challenging because computing gradients requires evaluating $ Z(\alpha, \beta)$.

👉 **Solution**: Instead of MLE, we use **Approximate Bayesian Computation (ABC)**.

------------------

In [25]:
from modules.ABC_reject import *
from modules.Gibbs_sampler import *

In [271]:
# Example run
n = 500  # Grid size
true_alpha, true_beta = 0.5, 1.2

observed_grid1 = run_gibbs(n, true_alpha, true_beta, steps=1)
# observed_grid = ising_model(n, true_alpha, true_beta)
obs_stats1 = sufficient_statistics(observed_grid1)

In [286]:
# Example run
n = 500  # Grid size
true_alpha, true_beta = 0.5, 1.2

observed_grid2 = run_gibbs(n, true_alpha, true_beta, steps=1)
# observed_grid = ising_model(n, true_alpha, true_beta)
obs_stats2 = sufficient_statistics(observed_grid2)

In [287]:
print(mse_distance(np.array(obs_stats1), np.array(obs_stats2)))
print(mae_distance(np.array(obs_stats1), np.array(obs_stats2)))

460544.5
631.5


In [288]:
# Example run
n = 500  # Grid size


theta_samples = abc_reject(
    obs_stats1,
    prior_alpha=(-1, 1),
    prior_beta=(0, 2),
    n=n,
    epsilon=2000,
    num_samples=1000,
)


print("Estimated alpha and beta:", np.mean(theta_samples, axis=0))

min_idx = np.argmin(theta_samples[:, 2])

# Get the corresponding alpha and beta
best_alpha, best_beta, best_dis = (
    theta_samples[min_idx, 0],
    theta_samples[min_idx, 1],
    theta_samples[min_idx, 2],
)

print("nearest alpha and beta: ", (best_alpha, best_beta))
print("best distance: ", best_dis)
print(
    " MSE :",
    0.5
    * (
        (0.5 - np.mean(theta_samples, axis=0)[0]) ** 2
        + abs(1.2 - np.mean(theta_samples, axis=0)[1]) ** 2
    )
    ** (0.5),
)

-0.9743917896572727 0.07319704677087246 124408
-0.49878833980913573 1.6239205813554407 89761
0.7546452426477541 1.1609345041449892 20667
0.9167533725166808 0.715737932049092 29092
0.4665489331869188 0.5893490045048158 11752
-0.04453622820236536 1.3663644877341052 49337
-0.24384037933436398 1.8236745591819319 66146
0.48803602760396925 0.6252066984514548 9432
0.9065801268274813 0.3422207042626142 32012
-0.028482843045227302 1.3399359137391647 47370
0.22350462543221816 1.3294973771767802 23709
-0.24599644417215472 1.9172074924166007 67353
-0.6776206831922282 1.5912041354494648 101983
-0.6013736478759084 0.40846135615477785 104898
0.0737996869155213 0.4230706037924745 54499
0.94100880865355 0.5176008534489696 32699
0.10270655592991851 0.9526201118511464 39081
0.8704229315347967 0.2708736668918772 31587
0.6233744651365902 1.9006907993141158 10109
-0.8864311674820256 1.4013446667671963 119031
0.5075459825279125 0.5196066873609135 11264
0.42378101289074377 1.4012284334471605 9395
0.2011593188

# Approximate Bayesian Computation (ABC-Reject) for Ising Model

## 1. Understanding the Sufficient Statistics

In the Ising model, we define the **sufficient statistics**:

$ S(x) = \sum_i x_i, \quad S_2(x) = \sum_{i \sim j} \mathbf{1}_{x_i = x_j} $

### What is $ S_2 $?
- It represents the number of **neighboring pairs** where the spins are the same.
- This is a key statistic because it captures the effect of the interaction parameter $ \beta $, which favors alignment.



## 2. Choosing the Distance Function

Since ABC-Reject compares **simulated** and **observed** data, we define a distance function:

$ d(S(x_{\text{obs}}), S(x_{\text{sim}})) = \left| S(x_{\text{obs}}) - S(x_{\text{sim}}) \right| + \left| S_2(x_{\text{obs}}) - S_2(x_{\text{sim}}) \right| $

### Why this choice?
- $ S(x) $ captures the **overall magnetization** (effect of $ \alpha $).
- $ S_2(x) $ captures **neighboring alignment** (effect of $ \beta $).
- The absolute difference ensures simple, interpretable comparison.
- A **small threshold $ \epsilon $** ensures we accept only close matches.



## 3. ABC-Reject Algorithm Explained

Since computing the likelihood function is intractable, **ABC** estimates parameters using simulation:

1. **Generate observed data** $ x_{\text{obs}} $ from the Ising model with **unknown** $ (\alpha^*, \beta^*) $.
2. **For many iterations**:
   - Sample $ (\alpha, \beta) $ from a **prior distribution**.
   - Simulate $ x_{\text{sim}} $ using the Ising model with $ (\alpha, \beta) $.
   - Compute sufficient statistics $ S_2(x_{\text{sim}}) $.
   - **Accept** $ (\alpha, \beta) $ if the distance is **less than** $ \epsilon $.
3. The set of **accepted** $ (\alpha, \beta) $ values approximates their posterior distribution.



## 4. Why Use ABC-Reject?

- **No need for likelihood computation** (which is intractable due to $ Z(\alpha, \beta) $).
- **Flexible**—can be applied to models with unknown normalizing constants.
- **Drawback**: Inefficient for high-dimensional problems (many rejections).

---


## Why are $ S(x) $ and $ S_2(x) $ sufficient statistics for the Ising Model?

For the Ising model, the parameters we want to estimate are $ \alpha $ and $ \beta $, which affect the magnetization and the interactions between neighboring spins, respectively.

- **Magnetization (summed spins):**

  $$
  S(x) = \sum_i x_i
  $$

  This statistic captures the overall magnetic alignment (effect of $ \alpha $) in the system.

- **Neighbor interactions:**

  $$
  S_2(x) = \sum_{i \sim j} \mathbf{1}_{x_i = x_j}
  $$

  This statistic counts how many neighboring spins are aligned (effect of $ \beta $).


### Intuition:

- The **magnetization statistic** $ S(x) $ captures how many spins are aligned, which is directly influenced by $ \alpha $ (external field).
- The **interaction statistic** $ S_2(x) $ captures how many neighboring spins are aligned, which is directly influenced by $ \beta $ (interaction strength).
- Together, these statistics summarize the system in a way that no additional information (about individual spins or the precise configuration) is needed to estimate $ \alpha $ and $ \beta $.






RESULTS for $\alpha$ = 0.5 and $\beta$ = 1.2 : 

| $ n $ (System Size) | Number of Samples | $ \epsilon $ (Tolerance) | $ \alpha $ Estimate | $ \beta $ Estimate | Time (s) | MSE |
|---------------------|-------------------|--------------------------|---------------------|--------------------|----------|-----|
| 10                  | 100000             | 3                      | 0.62                | 1.11               | 87.7        | 0.08 |
| 20                  | 100000              | 10                     | 0.68                | 1.32               | 306.6       | 0.10 |
| 30                  | 10000              | 50                     | 0.29                | 1.39               | 78.2       | 0.12 |
| 50                  | 10000             |100                      | 0.63                | 1.36               | 309.1        | 0.11 |
| 100                  | 10000             | 100                      | 0.49                | 1.31               | 761.8        | 0.06 |
| 500                  | 10000             | 2000                      | 0.50                | 1.16               | 1901.7        | 0.01 |
